<a href="https://colab.research.google.com/github/OCanSagbas/CSV-Document-Processes/blob/main/CSV_File_Modifying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
file_path = '/content/goodreads-export.csv'
goodreads_data = pd.read_csv(file_path)
goodreads_data.head(10)
#Panda kütüphanesini indirdikten sonra Google Collab içinden csv dosyasını okuma ve ilk 10 satırını yazdırma işlemi.

In [ ]:
columns_to_remove = ['Spoiler', 'Private Notes', 'Recommended For', 'Recommended By', 'Original Purchase Date', 'Original Purchase Location', 'BCID', 'ISBN13']
goodreads_data = goodreads_data.drop(columns=columns_to_remove)
goodreads_data.head(10)
#Columns_to_remove variable'ında yazan sütunları kaldırma işlemi.

In [ ]:
goodreads_data = goodreads_data.set_index('Book Id')
goodreads_data.head(10)
#Book Id sütununu index olarak ayarlama işlemi.

In [ ]:
import re
def fix_name(column):
    new_string = ""
    for element in column:
        pattern = re.compile(r"[A-Za-z., ]")
        match = re.search(pattern, element)
        if match:
            new_string += element
        else:
            continue
    return new_string

def new_name(input_value):
    values = input_value.split(",")
    surnames = values[0]
    names = values[1]

    # Kısaltılmış isimleri tutacak bir liste oluştur
    shortened_names_list = []

    # İsimlerin içindeki her kelimeyi dolaş ve kısaltılmış ismi oluştur
    pattern = re.compile(r"[A-Za-z.]+")
    for match in pattern.finditer(names):
        start, end = match.span()
        word = names[start:end]
        shortened_names_list.append(word[0].upper() + ".")

    # Kısaltılmış isimleri birleştir ve soyadı ekleyerek sonucu döndür
    shortened_names = " ".join(shortened_names_list) + " " + surnames
    return shortened_names


goodreads_data["Author l-f"] =  goodreads_data["Author l-f"].apply(fix_name)
goodreads_data["Author l-f"] =  goodreads_data["Author l-f"].apply(new_name)
goodreads_data.head(10)


In [ ]:
# Final sorusu 5
def additional_authors(authors):
    if pd.notna(authors):
        formatted_authors = []
        for name in authors.split(','):
            parts = name.strip().split()
            if len(parts) > 1:
                formatted_name = f'{parts[0][0]}. {" ".join(parts[1:])}'
                formatted_authors.append(formatted_name)
            else:
                formatted_authors.append(name)
        return ', '.join(formatted_authors)
    return authors
#Michael LeBoeuf, Mel Lindauer -> M. LeBoeuf, M. Lindauer
goodreads_data['Additional Authors'] = goodreads_data['Additional Authors'].apply(additional_authors)
goodreads_data.head(10)


In [ ]:
goodreads_data['Author'] = goodreads_data[['Author l-f', 'Additional Authors']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
# "Author l-f" ve "Additional Authors" sütunlarını birleştirip "Author" sütununa yerleştirme işlemi
goodreads_data = goodreads_data.drop(['Author l-f', 'Additional Authors'], axis=1)
# "Author l-f" ve "Additional Authors" sütunlarını silme işlemi
goodreads_data.head(10)

In [ ]:

goodreads_data['ISBN'].fillna(0, inplace=True)
goodreads_data['Number of Pages'].fillna(0, inplace=True)
# ISBN ve Number of Pages sütunlarındaki eksik değerlere 0 atama işlemi
goodreads_data.head(10)


In [ ]:
# Eğer Original Publication Year değeri boş değilse, Year Published'i Original Publication Year ile değiştirme işlemi
goodreads_data.loc[goodreads_data['Original Publication Year'].notnull(), 'Year Published'] = goodreads_data['Original Publication Year']
goodreads_data.head(10)

In [ ]:
# Bookshelves with positions sütununu düzenle
def extract_positions(value):
    matches = re.findall(r'#(\d+)', str(value))
    if matches:
        return ','.join(matches)
    return ''
goodreads_data['Bookshelves with positions'] = goodreads_data['Bookshelves with positions'].apply(extract_positions)
goodreads_data.head(50)


In [ ]:
# Tarih formatını yıla dönüştür
def extract_year(date):
    try:
        return pd.to_datetime(date, errors='coerce').year
    except:
        return date

# Date Read ve Date Added sütunlarını düzenle
goodreads_data['Date Read'] = goodreads_data['Date Read'].apply(extract_year)
goodreads_data['Date Added'] = goodreads_data['Date Added'].apply(extract_year)

# Veriyi kontrol etmek için ilk 10 satırı göster
goodreads_data.head(10)


In [ ]:
# Quickness sütununu oluştur
goodreads_data['Quickness'] = (goodreads_data['Date Read'] == goodreads_data['Date Added']).astype(int)

# Veriyi kontrol etmek için ilk 10 satırı göster
goodreads_data.head(10)



In [ ]:
import string
import numpy as np

# My Review sütunundaki metinlerin kelime sayısını güncelle
def count_words(review):
    # NaN kontrolü ekleyerek
    if isinstance(review, str):
        # Noktalama işaretlerini kaldır
        review = review.translate(str.maketrans('', '', string.punctuation))
        # Boşluklardan böl ve kelime sayısını döndür
        return len(review.split())
    else:
        return np.nan  # NaN değeri varsa NaN olarak bırak

# My Review sütununu güncelle
goodreads_data['My Review'] = goodreads_data['My Review'].apply(count_words)

# Sonuçları göster
goodreads_data.head(30)


In [ ]:
import matplotlib.pyplot as plt

# Bookshelves sütunundaki dolu değerleri filtrele
non_empty_bookshelves = goodreads_data['Bookshelves'].dropna()

# Dolu değerlerin yüzdelerini hesapla
shelf_percentages = non_empty_bookshelves.value_counts(normalize=True) * 100

# Pie Chart oluştur
plt.figure(figsize=(10, 8))
plt.pie(shelf_percentages, labels=shelf_percentages.index, autopct='%1.1f%%', startangle=140)
plt.title('Bookshelves Distribution')
plt.show()


In [ ]:
import seaborn as sns

# Year Published sütunundaki değerlerin histogramını oluştur
plt.figure(figsize=(12, 6))
sns.histplot(goodreads_data['Year Published'].dropna(), bins=range(int(goodreads_data['Year Published'].min()), int(goodreads_data['Year Published'].max()) + 1, 1), kde=False)
plt.title('Number of Books Published Each Year')
plt.xlabel('Year Published')
plt.ylabel('Number of Books')
plt.show()


In [ ]:
# Exclusive Shelf sütunundaki değerlerin sayısını hesapla
exclusive_shelf_counts = goodreads_data['Exclusive Shelf'].value_counts()

# Bar Graph (Çubuk Grafiği)
plt.figure(figsize=(12, 6))
sns.barplot(x=exclusive_shelf_counts.index, y=exclusive_shelf_counts.values)
plt.title('Distribution of Exclusive Shelf Values (Bar Graph)')
plt.xlabel('Exclusive Shelf')
plt.ylabel('Count')
plt.show()

# Pie Chart (Pasta Grafiği)
plt.figure(figsize=(8, 8))
plt.pie(exclusive_shelf_counts, labels=exclusive_shelf_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Exclusive Shelf Values (Pie Chart)')
plt.show()


In [ ]:
# NaN değerlere 0 ata
goodreads_data['Number of Pages'] = goodreads_data['Number of Pages'].fillna(0)

# Sayfa sayılarını tam sayıya çevir
goodreads_data['Number of Pages'] = goodreads_data['Number of Pages'].astype(int)

# Sayfa sayılarına göre histogram oluştur
plt.figure(figsize=(12, 6))
sns.histplot(goodreads_data['Number of Pages'], bins=range(0, max(goodreads_data['Number of Pages']) + 50, 50), kde=False)
plt.title('Distribution of Number of Pages')
plt.xlabel('Number of Pages')
plt.ylabel('Count')
plt.show()


In [ ]:
# Binding sütunundaki NaN değerlere 'Unknown' ata
goodreads_data['Binding'] = goodreads_data['Binding'].fillna('Unknown')

# Violin Plot çizin
plt.figure(figsize=(12, 8))
sns.violinplot(x='Binding', y='My Rating', data=goodreads_data)
plt.title('Violin Plot of My Rating by Binding')
plt.xlabel('Binding')
plt.ylabel('My Rating')
plt.show()


In [ ]:
# Binding sütunundaki NaN değerlere 'Unknown' ata
goodreads_data['Binding'] = goodreads_data['Binding'].fillna('Unknown')

# Violin Plot çizin
plt.figure(figsize=(12, 8))
sns.violinplot(x='Binding', y='Average Rating', data=goodreads_data)
plt.title('Violin Plot of Average Rating by Binding')
plt.xlabel('Binding')
plt.ylabel('Average Rating')
plt.show()


In [ ]:
# NaN değerleri temizle
ratings_data = goodreads_data[['My Rating', 'Average Rating']].dropna()

# Scatter plot çizin
plt.figure(figsize=(10, 6))
sns.scatterplot(x='My Rating', y='Average Rating', data=ratings_data)
plt.title('Scatter Plot between My Rating and Average Rating')
plt.xlabel('My Rating')
plt.ylabel('Average Rating')
plt.show()


In [ ]:
# NaN değerleri temizle
ratings_data = goodreads_data[['Number of Pages', 'My Rating']].dropna()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Number of Pages', y='My Rating', data=ratings_data)
plt.title('Seaborn ile Scatter Plot')
plt.xlabel('Number of Pages')
plt.ylabel('My Rating')
plt.show()


# Verileri seç
ratings_data = goodreads_data[['Number of Pages', 'My Rating']].dropna()
number_of_pages = ratings_data['Number of Pages']
my_rating = ratings_data['My Rating']

# Matplotlib ile Scatter Plot çizin
plt.figure(figsize=(10, 6))
plt.scatter(x=number_of_pages, y=my_rating, color='blue')
plt.title('Matplotlib ile Scatter Plot')
plt.xlabel('Number of Pages')
plt.ylabel('My Rating')
plt.show()


In [ ]:

print(goodreads_data.head())

# Düzenlenmiş verileri başka bir CSV dosyasına yazma
goodreads_data.to_csv('duzenlenmis_veriler.csv', index=False)
